In [1]:
# attention:
# install neded libs:
# python -m pip install --upgrade pip
# pip install --upgrade --user numpy scipy pandas matplotlib sklearn jupyter tensorflow-gpu keras opencv-python
# install CUDA for the GPU
# start with: 0. go to dir inside some terminal with proper PATH 1. "$ jupyter notebook"

import os
import cv2
import re
import random
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img
from keras.models import Sequential 
from keras import layers, models, optimizers, utils
from keras import backend as K
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "C:\Users\mpe\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "C:\Users\mpe\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "C:\Users\mpe\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "c:\users\mpe\appdata\local\programs\python\python37\lib\imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "c:\users\mpe\appdata\local\programs\python\python37\lib\imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: DLL load failed: Das angegebene Modul wurde nicht gefun

In [ ]:
img_width = 150
img_height = 150
# adapt those paths to the current directories
#C:\Users\mpe\Desktop\MarcelsFolder\coding\machineLearningWithPython\3rdSession\test1
TRAIN_DIR = 'C:/Users/mpe/Desktop/MarcelsFolder/coding/machineLearningWithPython/3rdSession/train/'
TEST_DIR = 'C:/Users/mpe/Desktop/MarcelsFolder/coding/machineLearningWithPython/3rdSession/test1/'

train_images_dogs_cats = [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR)] # use this for full dataset
test_images_dogs_cats = [TEST_DIR+i for i in os.listdir(TEST_DIR)]


In [ ]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]

In [ ]:
train_images_dogs_cats.sort(key=natural_keys)
train_images_dogs_cats = train_images_dogs_cats[0:1300] + train_images_dogs_cats[12500:13800] 

test_images_dogs_cats.sort(key=natural_keys)

In [ ]:
def prepare_data(list_of_images):
    """
    Returns two arrays: 
        x is an array of resized images
        y is an array of labels
    """
    x = [] # images as arrays
    y = [] # labels
    
    for image in list_of_images:
        x.append(cv2.resize(cv2.imread(image), (img_width,img_height), interpolation=cv2.INTER_CUBIC))
    
    for i in list_of_images:
        if 'dog' in i:
            y.append(1)
        elif 'cat' in i:
            y.append(0)
        #else:
            #print('neither cat nor dog name present in images')
            
    return x, y

In [ ]:
X, Y = prepare_data(train_images_dogs_cats)
print(K.image_data_format())

In [ ]:
# First split the data in two sets, 80% for training, 20% for Val/Test)
X_train, X_val, Y_train, Y_val = train_test_split(X,Y, test_size=0.2, random_state=1)

In [ ]:
nb_train_samples = len(X_train)
nb_validation_samples = len(X_val)
batch_size = 16

In [ ]:
model = Sequential()

model.add(layers.Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(32, (3, 3)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(64, (3, 3)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1))
model.add(layers.Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.summary()


In [ ]:
train_datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow(np.array(X_train)/255, Y_train, batch_size=batch_size)
validation_generator = val_datagen.flow(np.array(X_val)/255, Y_val, batch_size=batch_size)


In [ ]:
train_generator

In [ ]:
history = model.fit_generator(
    train_generator, 
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=500,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size
)

In [ ]:
#model.save_weights('model_wieghts.h5')
#model.save('model_keras.h5')
a = model.predict_classes(np.array(X_val)/255)
np.mean(np.reshape(np.array(Y_val),[-1,1]) == a)

In [ ]:
plt.plot(history.history['acc'],color = 'red')
plt.plot(history.history['val_acc'],color = 'blue')


In [ ]:
# Looking at some of the mistakes
y_pred = model.predict_classes(np.array(X_val)/255)
ind = np.array(np.where((np.reshape(np.array(Y_val),[-1,1]) != y_pred) & (y_pred == 0)))


In [ ]:
k = ind[0,2]
plt.imshow(X_val[k])
print(Y_val[k])
print(y_pred[k])